In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
parameters={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
           "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }

In [33]:
test = pd.read_csv("/content/drive/MyDrive/IDM/7mo semestre idm/Reto1/Modelo/test.csv")
id = test["Id"]

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/7 semestre/Reto1/Modelo/test.csv")
id = test["Id"]
test = test.drop(["Id", "LotFrontage"], axis=1)

test['GarageYrBlt'] = test['GarageYrBlt'].replace('No', np.nan)
test['GarageYrBlt'] = test['GarageYrBlt'].astype(float)
test['GarageYrBlt'] = test['GarageYrBlt'].fillna(0).astype(int)

test['Alley'] = test['Alley'].astype(str).str.strip().replace('nan', 'No')

test['MasVnrType'] = test['MasVnrType'].astype(str).str.strip().replace(['None', 'nan'], 'No')

test['BsmtQual'] = test['BsmtQual'].apply(lambda a:100 if a == 'Ex' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:95 if a == 'Gd' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:85 if a == 'TA' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:75 if a == 'Fa' else a)
test['BsmtQual'] = test['BsmtQual'].apply(lambda a:70 if a == 'Po' else a)
test["BsmtQual"] = test["BsmtQual"].fillna(0)
test['BsmtQual'] = test['BsmtQual']. astype(int)
test["BsmtCond"] = test["BsmtCond"].fillna("No")
test["BsmtExposure"] = test["BsmtExposure"].fillna("No")
test["BsmtFinType1"] = test["BsmtFinType1"].fillna("No")
test["BsmtFinType2"] = test["BsmtFinType2"].fillna("No")
test['FireplaceQu'] = test['FireplaceQu'].astype(str).str.strip().replace('nan', 'No')
test['GarageType'] = test['GarageType'].astype(str).str.strip().replace('nan', 'No')
test['GarageFinish'] = test['GarageFinish'].astype(str).str.strip().replace('nan', 'No')
test['GarageQual'] = test['GarageQual'].astype(str).str.strip().replace('nan', 'No')
test['PoolQC'] = test['PoolQC'].astype(str).str.strip().replace('nan', 'No')

# Op 1

op1 = test.copy()

# Label encoding
op1["Utilities"] = op1["Utilities"].replace({'AllPub': 4, 'NoSewr': 3,'NoSeWa': 2,'ELO': 1})
op1["ExterQual"] = op1["ExterQual"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0,})
op1["ExterCond"] = op1["ExterCond"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0,})
op1["BsmtCond"] = op1["BsmtCond"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["BsmtExposure"] = op1["BsmtExposure"].replace({'Gd': 3, 'Av': 2,'Mn': 1,'No': 0})
op1["BsmtFinType1"] = op1["BsmtFinType1"].replace({'GLQ': 6, 'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1,'No': 0})
op1["BsmtFinType2"] = op1["BsmtFinType2"].replace({'GLQ': 6, 'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1,'No': 0})
op1["HeatingQC"] = op1["HeatingQC"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0})
op1["CentralAir"] = op1["CentralAir"].replace({'Y': 1, 'N': 0})
op1["KitchenQual"] = op1["KitchenQual"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'Po': 0,})
op1["FireplaceQu"] = op1["FireplaceQu"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["GarageQual"] = op1["GarageQual"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["GarageCond"] = op1["GarageCond"].replace({'Ex': 5, 'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'No': 0})
op1["PoolQC"] = op1["PoolQC"].replace({'Ex': 4, 'Gd': 3,'TA': 2,'Fa': 1,'No': 0})



# Hot encoding
categorical_cols = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical','Functional', 'GarageType', 'GarageFinish','PavedDrive','Fence','MiscFeature','SaleType','SaleCondition']
op1 = pd.get_dummies(op1, columns=categorical_cols)
op1['RoofMatl_ClyTile'] = 0

# Estandarización
variables_no_estandarizar = ['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_No', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker', 'Condition1_Artery', 'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA', 'Condition1_PosN', 'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Artery', 'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl', 'RoofStyle_Flat', 'RoofStyle_Gable', 'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'RoofMatl_ClyTile', 'RoofMatl_CompShg', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_CBlock', 'Exterior1st_CemntBd', 'Exterior1st_HdBoard', 'Exterior1st_ImStucc', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_VinylSd', 'Exterior1st_Wd Sdng', 'Exterior1st_WdShing', 'Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_Brk Cmn', 'Exterior2nd_BrkFace', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_ImStucc', 'Exterior2nd_MetalSd', 'Exterior2nd_Other', 'Exterior2nd_Plywood', 'Exterior2nd_Stone', 'Exterior2nd_Stucco', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'Exterior2nd_Wd Shng', 'MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_No', 'MasVnrType_Stone', 'Foundation_BrkTil', 'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'Heating_Floor', 'Heating_GasA', 'Heating_GasW', 'Heating_Grav', 'Heating_OthW', 'Heating_Wall', 'Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr', 'Functional_Maj1', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ', 'GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageType_No', 'GarageFinish_Fin', 'GarageFinish_No', 'GarageFinish_RFn', 'GarageFinish_Unf', 'PavedDrive_N', 'PavedDrive_P', 'PavedDrive_Y', 'Fence_GdPrv', 'Fence_GdWo', 'Fence_MnPrv', 'Fence_MnWw', 'Fence_No', 'MiscFeature_Gar2', 'MiscFeature_No', 'MiscFeature_Othr', 'MiscFeature_Shed', 'MiscFeature_TenC', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'SaleCondition_Abnorml', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial']

scaler = StandardScaler()
columnas_a_estandarizar = [col for col in op1.columns if col not in variables_no_estandarizar]
op1[columnas_a_estandarizar] = scaler.fit_transform(op1[columnas_a_estandarizar])
op1_Kaggle = pd.DataFrame(op1)

# Eliminación de columnas
op1_Kaggle = op1_Kaggle.drop(['FireplaceQu', 'GarageYrBlt', 'GarageCond', 'GarageArea', 'PoolArea', 'MiscFeature_Gar2',
                                  'Neighborhood_Somerst', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard',
                                  'Exterior2nd_MetalSd', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng',
                                  'SaleCondition_Partial'], axis=1)



# Selección de variables significantes
significant_variables_list = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtExposure', 'BsmtFinSF1',
                              'TotalBsmtSF', 'HeatingQC', 'CentralAir', 'GrLivArea', 'BsmtFullBath', 'HalfBath', 'KitchenAbvGr',
                              'KitchenQual', 'Fireplaces', 'GarageCars', 'GarageQual', 'WoodDeckSF', 'ScreenPorch', 'MSZoning_FV',
                              'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'LotConfig_Inside', 'LandSlope_Sev', 'Neighborhood_Crawfor',
                              'Neighborhood_Edwards', 'Neighborhood_MeadowV', 'Neighborhood_NridgHt', 'Neighborhood_StoneBr',
                              'Condition1_Artery', 'Condition1_Feedr', 'Condition1_RRAe', 'Condition2_PosA', 'Condition2_PosN',
                              'BldgType_Twnhs', 'BldgType_TwnhsE', 'RoofMatl_ClyTile', 'RoofMatl_CompShg', 'Exterior1st_BrkComm',
                              'Exterior1st_BrkFace', 'Exterior1st_MetalSd', 'Exterior2nd_BrkFace', 'Foundation_BrkTil',
                              'Foundation_CBlock', 'Foundation_Wood', 'Heating_GasA', 'Heating_Grav', 'Electrical_SBrkr',
                              'Functional_Maj1', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Typ',
                              'GarageType_2Types', 'SaleType_COD', 'SaleType_ConLI', 'SaleType_WD', 'SaleCondition_Normal']

op1_Kaggle = op1_Kaggle[significant_variables_list]

In [ ]:
# Op 2

op2 = test.copy()

# Hot encoding
categoricas = op2.select_dtypes(include='object').columns
numericas = op2.select_dtypes(include='number').columns

categoricas = list(categoricas)
op2 = pd.get_dummies(op2, columns=categoricas)

# Estandarización
variables_no_estandarizar = ['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
                             'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl',
                             'Alley_No', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2',
                             'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS',
                             'LandContour_Low', 'LandContour_Lvl', 'LotConfig_Corner',
                             'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl',
                             'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
                             'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
                             'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
                             'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU',
                             'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber',
                             'Neighborhood_Veenker', 'Condition1_Artery', 'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA', 'Condition1_PosN',
                             'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Artery', 'Condition2_Feedr',
                             'Condition2_Norm', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'BldgType_1Fam',
                             'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf',
                             'HouseStyle_1Story', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl',
                             'RoofStyle_Flat', 'RoofStyle_Gable', 'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'RoofMatl_ClyTile',
                             'RoofMatl_CompShg', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_CBlock', 'Exterior1st_CemntBd', 'Exterior1st_HdBoard', 'Exterior1st_ImStucc', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_VinylSd', 'Exterior1st_Wd Sdng', 'Exterior1st_WdShing', 'Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_Brk Cmn', 'Exterior2nd_BrkFace', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_ImStucc', 'Exterior2nd_MetalSd', 'Exterior2nd_Other', 'Exterior2nd_Plywood', 'Exterior2nd_Stone', 'Exterior2nd_Stucco', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'Exterior2nd_Wd Shng', 'MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_No', 'MasVnrType_Stone', 'Foundation_BrkTil', 'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'Heating_Floor', 'Heating_GasA', 'Heating_GasW', 'Heating_Grav', 'Heating_OthW', 'Heating_Wall', 'Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr', 'Functional_Maj1', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ', 'GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageType_No', 'GarageFinish_Fin', 'GarageFinish_No', 'GarageFinish_RFn', 'GarageFinish_Unf', 'PavedDrive_N', 'PavedDrive_P', 'PavedDrive_Y', 'Fence_GdPrv', 'Fence_GdWo', 'Fence_MnPrv', 'Fence_MnWw', 'Fence_No', 'MiscFeature_Gar2', 'MiscFeature_No', 'MiscFeature_Othr', 'MiscFeature_Shed', 'MiscFeature_TenC', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'SaleCondition_Abnorml', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial']


scaler = StandardScaler()
columnas_a_estandarizar = [col for col in op2.columns if col not in variables_no_estandarizar]
op2[columnas_a_estandarizar] = scaler.fit_transform(op2[columnas_a_estandarizar])
op2_Kaggle = pd.DataFrame(op2)

# Eliminación de columnas
op2['Condition2_RRAn'] = 0
op2['Condition2_RRNn'] = 0
op2['HouseStyle_2.5Fin'] = 0
op2['RoofMatl_Membran'] = 0
op2['RoofMatl_Metal'] = 0
op2['RoofMatl_Roll'] = 0
op2['Electrical_Mix'] = 0
op2['GarageQual_Ex'] = 0


op2_Kaggle = op2_Kaggle.drop(['Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_MetalSd',
                              'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'GarageCars', 'SaleCondition_Partial', 'MiscFeature_Gar2',
                              'Neighborhood_Somerst', 'BsmtFinType1_No', 'BsmtFinType2_No',  'GarageFinish_No', 'GarageQual_No'], axis=1)

# Selección de variables significantes
significant_variables_list = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF',
                              'GrLivArea', 'BsmtFullBath', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageArea', 'WoodDeckSF',
                              'EnclosedPorch', 'ScreenPorch', 'PoolArea', 'MSZoning_C (all)', 'MSZoning_RM', 'Street_Pave',
                              'LandContour_Bnk', 'LandContour_Low', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Sev',
                              'Neighborhood_BrDale', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
                              'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NWAmes',
                              'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW',
                              'Neighborhood_StoneBr', 'Condition1_Artery', 'Condition1_Feedr', 'Condition1_RRAe', 'Condition2_Artery',
                              'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosA', 'Condition2_RRAn', 'Condition2_RRNn',
                              'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_2.5Fin', 'RoofStyle_Flat', 'RoofStyle_Gable',
                              'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofMatl_CompShg', 'RoofMatl_Membran',
                              'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl',
                              'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_MetalSd', 'Exterior2nd_BrkFace', 'ExterCond_Gd',
                              'Foundation_BrkTil', 'Foundation_CBlock', 'Foundation_Wood', 'BsmtCond_Po', 'BsmtExposure_Gd', 'Heating_GasA',
                              'Heating_Grav', 'HeatingQC_Ex', 'CentralAir_N', 'Electrical_Mix', 'KitchenQual_Ex', 'Functional_Maj1',
                              'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Typ', 'GarageType_Attchd',
                              'GarageType_Detchd', 'GarageQual_Ex', 'GarageCond_Ex', 'GarageCond_Fa', 'PoolQC_No', 'Fence_GdWo',
                              'SaleType_COD', 'SaleType_ConLI', 'SaleType_WD', 'SaleCondition_Normal']

op2_Kaggle = op2_Kaggle[significant_variables_list]

<ipython-input-11-76af8e706133>:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  op2['Condition2_RRAn'] = 0
<ipython-input-11-76af8e706133>:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  op2['Condition2_RRNn'] = 0
<ipython-input-11-76af8e706133>:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  o

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Guardar el DataFrame en un archivo CSV
op1_Kaggle.to_csv('/content/drive/MyDrive/7 semestre/Reto1/Modelo/test_final/op1_Kaggle.csv', index=False)
op2_Kaggle.to_csv('/content/drive/MyDrive/7 semestre/Reto1/Modelo/test_final/op2_Kaggle.csv', index=False)

NameError: ignored

# Label Encoding + One Hot Encoding

In [5]:
df_label = pd.read_csv("/content/drive/MyDrive/IDM/7mo semestre idm/Reto1/Modelo/dflabelstand.csv")

In [6]:
df_label.head()

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtExposure,BsmtFinSF1,TotalBsmtSF,HeatingQC,CentralAir,...,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Typ,GarageType_2Types,SaleType_COD,SaleType_ConLI,SaleType_WD,SaleCondition_Normal,SalePriceLog
0,-0.207142,0.651479,-0.517200,1.050994,0.878668,-0.631676,0.575425,-0.459303,0.891179,0.263813,...,0,0,0,1,0,0,0,1,1,12.247694
1,-0.091886,-0.071836,2.179628,0.156734,-0.429577,2.256363,1.171992,0.466465,0.891179,0.263813,...,0,0,0,1,0,0,0,1,1,12.109011
2,0.073480,0.651479,-0.517200,0.984752,0.830215,0.331004,0.092907,-0.313369,0.891179,0.263813,...,0,0,0,1,0,0,0,1,1,12.317167
3,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.631676,-0.499274,-0.687324,-0.151386,0.263813,...,0,0,0,1,0,0,0,1,0,11.849398
4,0.375148,1.374795,-0.517200,0.951632,0.733308,1.293683,0.463568,0.199680,0.891179,0.263813,...,0,0,0,1,0,0,0,1,1,12.429216


In [7]:
# Data Split
X = df_label.drop("SalePriceLog", axis = 1)
y = df_label["SalePriceLog"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
regressionTree_Label = DecisionTreeRegressor()

In [9]:
search_RT_Label = GridSearchCV(regressionTree_Label,param_grid=parameters,scoring='neg_mean_squared_error',cv=5)

In [10]:
search_RT_Label.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
28000 fits failed out of a total of 63000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7000 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [1, 3, 5, 7, 9, 11, 12],
                         'max_leaf_nodes': [None, 10, 20, 30, 40, 50, 60, 70,
                                            80, 90],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_weight_fraction_leaf': [0.1, 0.2, 0.3, 0.4, 0.5,
                                                      0.6, 0.7, 0.8, 0.9],
                         'splitter': ['best', 'random']},
             scoring='neg_mean_squared_error')

In [11]:
search_RT_Label.best_params_

{'max_depth': 11,
 'max_leaf_nodes': 70,
 'min_samples_leaf': 3,
 'min_weight_fraction_leaf': 0.1,
 'splitter': 'best'}

In [12]:
search_RT_Label.best_score_

-0.05433916887481827

**Best params:**

{'max_depth': 9,
 'max_leaf_nodes': 50,
 'min_samples_leaf': 9,
 'min_weight_fraction_leaf': 0.1,
 'splitter': 'best'}

**Score:**

 -0.05433916887481829

In [13]:
regressionTree_LabelBest = DecisionTreeRegressor(splitter = search_RT_Label.best_params_.get("splitter"),
                                                 max_depth = search_RT_Label.best_params_.get("max_depth"),
                                                 min_samples_leaf = search_RT_Label.best_params_.get("min_samples_leaf"),
                                                 min_weight_fraction_leaf = search_RT_Label.best_params_.get("min_weight_fraction_leaf"),
                                                 max_leaf_nodes = search_RT_Label.best_params_.get("max_leaf_nodes"))

In [14]:
regressionTree_LabelBest.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=11, max_leaf_nodes=70, min_samples_leaf=3,
                      min_weight_fraction_leaf=0.1)

In [15]:
y_pred = np.exp(regressionTree_LabelBest.predict(X_test))
y_test = np.exp(y_test)

In [16]:
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE: 31335.75976411326
MSE: 2754815646.350442
RMSE: 52486.33771135534


# One Hot Encoding

In [17]:
df_hot = pd.read_csv("/content/drive/MyDrive/IDM/7mo semestre idm/Reto1/Modelo/dfhotstand.csv")

In [18]:
df_hot.head()

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,TotalBsmtSF,GrLivArea,BsmtFullBath,KitchenAbvGr,...,GarageQual_Ex,GarageCond_Ex,GarageCond_Fa,PoolQC_No,Fence_GdWo,SaleType_COD,SaleType_ConLI,SaleType_WD,SaleCondition_Normal,SalePriceLog
0,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.575425,-0.459303,0.370333,1.107810,-0.211454,...,-0.045376,-0.037037,-0.156721,0.069409,0,0,0,1,1,12.247694
1,-0.091886,-0.071836,2.179628,0.156734,-0.429577,1.171992,0.466465,-0.482512,-0.819964,-0.211454,...,-0.045376,-0.037037,-0.156721,0.069409,0,0,0,1,1,12.109011
2,0.073480,0.651479,-0.517200,0.984752,0.830215,0.092907,-0.313369,0.515013,1.107810,-0.211454,...,-0.045376,-0.037037,-0.156721,0.069409,0,0,0,1,1,12.317167
3,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.499274,-0.687324,0.383659,1.107810,-0.211454,...,-0.045376,-0.037037,-0.156721,0.069409,0,0,0,1,0,11.849398
4,0.375148,1.374795,-0.517200,0.951632,0.733308,0.463568,0.199680,1.299326,1.107810,-0.211454,...,-0.045376,-0.037037,-0.156721,0.069409,0,0,0,1,1,12.429216


In [19]:
# Data Split
X = df_hot.drop("SalePriceLog", axis = 1)
y = df_hot["SalePriceLog"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
regressionTree_Hot =DecisionTreeRegressor()

In [21]:
search_RT_Hot = GridSearchCV(regressionTree_Hot,param_grid=parameters,scoring='neg_mean_squared_error',cv=5)

In [22]:
search_RT_Hot.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
28000 fits failed out of a total of 63000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7000 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [1, 3, 5, 7, 9, 11, 12],
                         'max_leaf_nodes': [None, 10, 20, 30, 40, 50, 60, 70,
                                            80, 90],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_weight_fraction_leaf': [0.1, 0.2, 0.3, 0.4, 0.5,
                                                      0.6, 0.7, 0.8, 0.9],
                         'splitter': ['best', 'random']},
             scoring='neg_mean_squared_error')

In [23]:
search_RT_Hot.best_params_

{'max_depth': 7,
 'max_leaf_nodes': 30,
 'min_samples_leaf': 2,
 'min_weight_fraction_leaf': 0.1,
 'splitter': 'best'}

In [24]:
search_RT_Hot.best_score_

-0.05484244488631492

**Best params:**

{'max_depth': 5,
 'max_leaf_nodes': 70,
 'min_samples_leaf': 6,
 'min_weight_fraction_leaf': 0.1,
 'splitter': 'best'}

**Score:**

-0.054842444886314956

In [25]:
regressionTree_HotBest = DecisionTreeRegressor(splitter = search_RT_Hot.best_params_.get("splitter"),
                                                 max_depth = search_RT_Hot.best_params_.get("max_depth"),
                                                 min_samples_leaf = search_RT_Hot.best_params_.get("min_samples_leaf"),
                                                 min_weight_fraction_leaf = search_RT_Hot.best_params_.get("min_weight_fraction_leaf"),
                                                 max_leaf_nodes = search_RT_Hot.best_params_.get("max_leaf_nodes"))

In [26]:
regressionTree_HotBest.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=7, max_leaf_nodes=30, min_samples_leaf=2,
                      min_weight_fraction_leaf=0.1)

In [27]:
y_pred = np.exp(regressionTree_HotBest.predict(X_test))
y_test = np.exp(y_test)

In [28]:
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE: 31335.759764113165
MSE: 2754815646.350438
RMSE: 52486.33771135531


**Predicción Kaggle**

In [29]:
op1_Kaggle = pd.read_csv("/content/drive/MyDrive/IDM/7mo semestre idm/Reto1/Modelo/test_final/test_op1_Kaggle.csv")
op2_Kaggle = pd.read_csv("/content/drive/MyDrive/IDM/7mo semestre idm/Reto1/Modelo/test_final/test_op2_Kaggle.csv")

In [30]:
y_pred_Kaggle1 = np.exp(regressionTree_LabelBest.predict(op1_Kaggle))
y_pred_Kaggle2 = np.exp(regressionTree_HotBest.predict(op2_Kaggle))

In [34]:
id

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [38]:
dict1 = {"Id": id, "SalePrice":y_pred_Kaggle1}
kaggle1 = pd.DataFrame.from_dict(dict1)

dict2 = {"Id": id, "SalePrice":y_pred_Kaggle2}
kaggle2 = pd.DataFrame.from_dict(dict2)

In [41]:
# Guardar el DataFrame en un archivo CSV
kaggle1.to_csv('/content/drive/MyDrive/IDM/7mo semestre idm/Reto1/Modelo/test_final/regressionTree_Label.csv', index=False)
kaggle2.to_csv('/content/drive/MyDrive/IDM/7mo semestre idm/Reto1/Modelo/test_final/regressionTree_Hot.csv', index=False)

In [47]:
kaggle2

,Id,SalePrice
0,1461,122151.367397
1,1462,145143.658767
2,1463,143702.195047
3,1464,175522.771325
4,1465,294189.136843
...,...,...
1454,2915,97487.935372
1455,2916,97487.935372
1456,2917,145143.658767
1457,2918,122151.367397
